In [ ]:
# Hapus saja Library Yang Tidak Dipakai

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np
from math import sqrt
from numpy import log
from pandas import Series
from sympy import symbols, hessian, Function, N

import statsmodels as sm

# matplotlib and seaborn are used for plotting graphs
import matplotlib.pyplot as plt
from matplotlib.dates import date2num
import seaborn as sns
from datetime import datetime
import subprocess

In [ ]:
data1 = pd.read_excel('DATA-SVR.xlsx')
data1 = data1.dropna(axis=1, how='all')
data1.head()

In [ ]:
# Ini nanti digunakan buat menghitung jarak antar data latih di fungsi kernel RBF
X_jarak =data1.iloc[:,1:7]
X_jarak

# Split Dataset

In [ ]:
# Split dataset
X = data1.iloc[:,2:8]
Y = data1['Selisih']
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, shuffle=False)
X_train.shape[0]

# Matriks Hessian Polynomial dan Sequential Learning

In [ ]:
# Optimasi Pakai Grid Search
# Parameter SVR dan Parameter Fungsi kernel Polynomial (d)

lamda   = 0.01
epsilon = 0.00001
cLR     = 0.005
C       = 0.5
d       = 2      # Parameter Polynomial

# Bentuk Matriks Hessian

In [ ]:
## Matriks Hessian Polynomial

def matrixHessianPolynomial(X_train,lamda):
    xT = X_train.T
    # Rumus Sesuai Fungsi Kernel Polynomial (Pers 3.11)
    hessian_x_poly = (np.dot(X_train, xT) + 1) **d + lamda**2  
    return hessian_x_poly

hessian_x_poly = matrixHessianPolynomial(X_train,lamda)
#print(hessian_x_poly)
hessian_x_poly

# Sequential Learning

In [ ]:
######debug
a_star = list(np.zeros(X_train.shape[0]))
a = list(np.zeros(X_train.shape[0]))
kolom = hessian_x_poly.shape[0]
nilaiE = np.zeros((kolom,kolom))
total_error = []
for i in range(kolom):
    sum_a_star = 0
    for j in range(kolom):
        sum_a_star = (a_star[j] - a[j]) * hessian_x_poly[i][j]
    nilaiE = y_train[i] - sum_a_star
    total_error.append(nilaiE)
total_error[0]

In [ ]:
## Sequential Learning 
## Tujuan dari Seq.Learning untuk memperoleh nilai a_star dan a

maxR = max(hessian_x_poly.diagonal())
lr = cLR/maxR
lr
# def nilaiError_lin(hessian_linear,a_star,a,y_train):
#   a_star = list(np.zeros(X_train.shape[0]))
#   a = list(np.zeros(X_train.shape[0]))
#   kolom = hessian_linear.shape[0]
#   nilaiE = np.zeros((kolom,kolom))
#   total_error = []
#   for i in range(kolom):
#       sum_a_star = 0
#       for j in range(kolom):
#           sum_a_star = (a_star[j] - a[j]) * hessian_linear[i][j]
#       nilaiE = y_train[i] - sum_a_star
#       total_error.append(nilaiE)
#   #print(len(total_error))
#       return total_error

max_iteration = 100
a_star = list(np.zeros(X_train.shape[0]))
a_star_float = np.asarray(a_star)
a = list(np.zeros(X_train.shape[0]))
a_float = np.asarray(a)
nilaiEr = []
delta_a_star = []
nilaiErArray = []
fmax = 0

# for i in range(max_iteration):
    
#   nilaiEr.append(nilaiError_lin(hessian_linear, a_star[i], a, y_train))

nilaiErArray = (np.array(total_error))

delta_a_star = []
delta_a = []
a_final = []
a_star_final = []
fmax = []
for i in range(max_iteration):
  fmax.append(max(((lr*nilaiErArray[i]) - epsilon),-a_star_float[i]))
for i in range(max_iteration):
  delta_a_star.append(min(fmax[i],(C - a_star_float[i])))
  a_star_final.append(a_star_float[i] + delta_a_star[i]) 
a_maks = [] 
for i in range(max_iteration) :
    a_maks.append(max(lr*(-nilaiErArray[i]-epsilon),-a_float[i]))
for i in range(max_iteration):
    delta_a.append(min(a_maks[i],C - a_float[i]))
    a_final.append(delta_a[i] + a_float[i])
print("a star",a_star_final[0]) 
print("a_maks",a_final[0])  
plt.plot(a_star_final)  

In [ ]:
## FUNGSI REGRESI ADA DI (PERS 3.8)

In [ ]:
## Prediksi nilai baru 

def matrixHessianLinear_test(X_test, X_train,lamda):
    xT = X_train.T
    hessian_X_test = np.dot(X_test, xT) + lamda**2
    return hessian_X_test

hessian_X_test = matrixHessianLinear_test(X_test,X_train,lamda)
hessian_X_test_ = np.asarray(hessian_X_test)
# Fungsi Regresi
## Pakai data train bisa, data test gak bisa
baris = hessian_X_test.shape[0]
kolom = hessian_X_test.shape[1]
sum_fx = []
selisih_a = []
for i in range(len(a_star_final)):
  selisih_a.append(a_star_final[i] - a_star[i])
selisih_a = np.array(selisih_a[:77])
for i in range(baris):
  sum_ = 0
  for j in range(kolom):
    sum_ = selisih_a * (hessian_X_test_[i][j] * (lamda**2))
print(len(selisih_a))

#     sum_fx.append(sum((a_star_final[j] - a_final[j]) * (hessian_X_test_[i][j] * (lamda**2) )))   
# sum_fx = fungsiRegresi(hessian_X_test,a_star,a)
# sum_fx
plt.plot(sum_)

In [ ]:
plt.plot(y_test)
print(len(y_test))

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error 
mape = mean_absolute_percentage_error(y_test,sum_)
print("mape = %.2f" % (mape),"%")
